Writing the mlpf code  in tensorflow

In [47]:
import bz2, pickle
import numpy as np
import tensorflow as tf
import tensorflow.nn as nn
import sklearn
import sklearn.metrics
import matplotlib.pyplot as plt

In [2]:
import os.path as osp
import os
import sys
from glob import glob


In [3]:
import sys
sys.path += ['/home/sraj/UCSD/particleflow/mlpf/']

import tfmodel

In [4]:
!wget --no-check-certificate -nc https://zenodo.org/record/4452283/files/tev14_pythia8_ttbar_0_0.pkl.bz2


File ‘tev14_pythia8_ttbar_0_0.pkl.bz2’ already there; not retrieving.



In [5]:
data = pickle.load(bz2.BZ2File("tev14_pythia8_ttbar_0_0.pkl.bz2", "r"))


In [6]:
## 100 events in one file
len(data['X']), len(data['ygen'])

(100, 100)

In [7]:
## Pad the number of elements to a size that's divisible by the bins
Xs = []
ys = []

max_size = 50*128

for i in range(len(data['X'])):
    X = data['X'][i][:max_size, :]
    y = data['ygen'][i][:max_size, :]
    Xpad = np.pad(X,[(0, max_size - X.shape[0]), (0,0)])
    ypad = np.pad(y, [(0, max_size - y. shape[0]), (0,0)])
    Xpad = Xpad.astype(np.float32)
    ypad = ypad.astype(np.float32)
    Xs.append(Xpad)
    ys.append(ypad)
    
X = np.stack(Xs)
y = np.stack(ys)
    

## Defining a tensorflow FCN class for particleflow
Converting the pytorcgh into tensorflow, using this method which can be found using this link. link can be found [here](https://stackoverflow.com/questions/69148722/what-is-the-alias-to-pytorch-nn-module-in-tensorflow)

    

In [38]:
class MLPF_FCN(tf.keras.layers.Layer):
    def __init__(self, input_dim = 12, hidden_dim = 2, embedding_dim=2, output_dim = 2):
        super (MLPF_FCN, self).__init__()
        
        self.act = tf.nn.relu
        
        self.nn1 = tf.keras.Sequential(
            tf.keras.layers.Dense(input_dim, activation="relu"),
            tf.keras.layers.BatchNormalization(hidden_dim),
            self.act(),
            tf.keras.layers.Dense(hidden_dim, hidden_dim),
            tf.keras.BatchNormalization(hidden_dim),
            self.act(),
            tf.keras.layers.Dense(hidden_dim, embedding_dim),
        )
        
        self.nn2 = tf.keras.Sequential(
            tf.keras.layers.Dense(input_dim+embedding_dim),
            self.act(),
            tf.keras.layers.Dense(hidden_dim, output_dim),
        
        )
        
    def forward(self, X):
        embedding = self.nn1(X)
        return self.nn2(tf.concat([X, embedding], axis=1)), _, _
            
                    

In [39]:
## defining the tensorflow GNN class for particleflow
import pickle as pkl
import os.path as osp
import os
import sys
from glob import glob

from typing import Optional, Union
from tensorflow.keras.layers import Conv2D

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [46]:
class MLPF_GNN(tf.keras.layers.Layer):
    """
    GNN model based on Gravnet...

    Forward pass returns
        preds: tensor of predictions containing a concatenated representation of the pids and p4
        A: dict() object containing adjacency matrices for each message passing
        msg_activations: dict() object containing activations before each message passing
    """
    
    def __init__(self, input_dim =12, output_dim_id = 6, output_dim_p4 = 6, 
                embedding_dim = 2, hiddn_dim1 = 2, hidden_dim2 = 2, 
                num_convs=2, space_dim = 4, propagate_dim=2, k=8):
        super(MLPF_GNN, self).__init__()
        
        self.act = tf.nn.elu
        
        #(1) embedding
        
        self.nn1 = tf.keras.Sequential(
            tf.keras.layers.Dense(input_dim),
            self.act(),
            tf.keras.layers.Dense(hiddn_dim1, hiddn_dim1),
            self.act(),
            tf.keras.layers.Dense(hiddn_dim1, hiddn_dim1),
            self.act(),
            tf.keras.layers.Dense(hiddn_dim1, embedding_dim),
        )
        
        self.conv = tf.keras.layers.Layer()
        for i in range(num_convs):
            slef.conv.append(GravNetConv_LRP(embedding_dim, embedding_dim, space_dim, propagate_dim, k))
            
            
        ##(3) DNN layer: classifying pid 
        self.nn2 =  tf.keras.Sequential(
            tf.keras.layers.Dense(input_dim+embedding_dim),
            self.act(),
            tf.keras.layers.Dense(hiddn_dim2, hiddn_dim2),
            self.act(),
            tf.keras.layers.Dense(hiddn_dim2, hiddn_dim2),
            self.act(),
            tf.keras.layers.Dense(hiddn_dim2, embedding_dim),      
        )
            
       # (4) DNN layer: regressing p4
        self.nn3 = tf.keras.Sequentiall(
            tf.keras.layers.Dense(input_dim + output_dim_id),
            self.act(),
            tf.keras.layers.Dense(hidden_dim2, hidden_dim2),
            self.act(),
            tf.keras.layers.Dense(hidden_dim2, hidden_dim2),
            self.act(),
            tf.keras.layers.Dense(hidden_dim2, output_dim_p4),
        )
    
    
    
    def forward(self, batch):
        x0 = batch.x
        
        # embed the inputs
        embedding = self.nn1(x0)
        
        # perform a series of graph convolutions
        A = {}
        msg_activations ={}
        for num, conv in enumerate(self.conv):
            embedding, A[f'conv.{num}'], msg_activations[f'conv.{num}'] = conv(embedding)
        
        # predict the pid's
        preds_id = self.nn2(tf.concat([x0, embedding], axis=-1))
        
        # predict the p4's
        preds_p4 = self.nn3(tf.concat([x0, preds_id], axis=-1))
        
        return tf.concat([preds_id, preds_p4], axis= -1), A,  msg_activations         

**Defining the message passing here:**

In [41]:
import tensorflow as tf


def message_passing(nodes, edges, edge_features, message_fn, edge_keep_prob=1.0):
    """
    Pass messages between nodes and sum the incoming messages at each node.
    Implements equation 1 and 2 in the paper, i.e. m_{.j}^t &= \sum_{i \in N(j)} f(h_i^{t-1}, h_j^{t-1})
    :param nodes: (n_nodes, n_features) tensor of node hidden states.
    :param edges: (n_edges, 2) tensor of indices (i, j) indicating an edge from nodes[i] to nodes[j].
    :param edge_features: features for each edge. Set to zero if the edges don't have features.
    :param message_fn: message function, will be called with input of shape (n_edges, 2*n_features + edge_features). The output shape is (n_edges, n_outputs), where you decide the size of n_outputs
    :param edge_keep_prob: The probability by which edges are kept. Basically dropout for edges. Not used in the paper.
    :return: (n_nodes, n_output) Sum of messages arriving at each node.
    """
    n_nodes = tf.shape(nodes)[0]
    n_features = nodes.get_shape()[1].value
    n_edges = tf.shape(edges)[0]

    message_inputs = tf.gather(nodes, edges)  # n_edges, 2, n_features
    reshaped = tf.concat([tf.reshape(message_inputs, (-1, 2 * n_features)), edge_features], 1)
    messages = message_fn(reshaped)  # n_edges, n_output
    messages = tf.nn.dropout(messages, edge_keep_prob, noise_shape=(n_edges, 1))

    updates = tf.unsorted_segment_sum(messages, edges[:, 1], n_nodes)

    return updates

In [42]:
def GravNet_Conv_LRP(MessagePassing):
    """
    Copied from pytorch_geometric source code, with the following edits
      a. retrieve adjacency matrix (we call A), and the activations before the message passing step (we call msg_activations)
      b. switched the execution of self.lin_s & self.lin_p so that the message passing step can substitute out of the box self.lin_s for lrp purposes
      c. used reduce='sum' instead of reduce='mean' in the message passing
      d. removed skip connection
    """
    def __init__(self, in_channels: int, out_channels: int, 
                space_dimensions: int, propagate_dimensions: int, k:int,
                num_workers: int = 1, **kwargs):
        super().__init__(flow = 'source_to_target', **kwargs)
        
        if knn is None:
            raise ImportError('`GravNetConv` not properly defined.')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.k = k
        self.num_workers = num_workers

        self.lin_p = Linear(in_channels, propagate_dimensions)
        self.lin_s = Linear(in_channels, space_dimensions)
        self.lin_out = Linear(propagate_dimensions, out_channels)

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_s.reset_parameters()
        self.lin_p.reset_parameters()
        self.lin_out.reset_parameters()

    def forward(
            self, x: Union[Tensor, PairTensor],
            batch: Union[OptTensor, Optional[PairTensor]] = None) -> Tensor:
        """"""

        is_bipartite: bool = True
        if isinstance(x, Tensor):
            x: PairTensor = (x, x)
            is_bipartite = False

        if x[0].dim() != 2:
            raise ValueError("Static graphs not supported in 'GravNetConv'")

        b: PairOptTensor = (None, None)
        if isinstance(batch, Tensor):
            b = (batch, batch)
        elif isinstance(batch, tuple):
            assert batch is not None
            b = (batch[0], batch[1])
            
        # embed the inputs before message passing
        msg_activations = self.lin_p(x[0])
        
        #Transform to space dimension to build the graph
        s_l: Tensor = self.lin_s(x[0])
        s_r: Tensor = self.lin_s(x[1]) if is_bipartite else s_l
            
        edge_index = knn(s_l, s_r, self.k, b[0], b[1]).flip([0])
        
        edge_weight = (s_l[edge_index[0]] - s_r[edge_index[1]]).pow(2).sum(-1)
        edge_weight = tf.exp(-10. * edge_weight)   
        
        # return the adjecency matrix
        A = tf.nn.layer.graph_convolution.feature_steered_convolution_layer (edge_index.to('cpu'), edge_attr=edge_weight.to('cpu'))[0]
        
        #message passing
        
        out = self.propagate(edge_index, x = (msg_activations, None),
                             edge_weight=edge_weight,
                             size = (s_l.size(0), s_r.size(0)))
        return self.lin_out(out), A, msg_activations
    
    def message(self, x_j: Tensor, index: Tensor)-> Tensor:
        return x_j * edge_weight.unsqueeze(1)
    
    def aggregate(self, inputs: Tensor, index: Tensor,
                  dim_size: Optional[int] = None) -> Tensor:
        out_mean = scatter(inputs, index, dim=self.node_dim, dim_size=dim_size,
                           reduce='sum')
        return out_mean

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels}, k={self.k})')

***REF* for GCN** 

1. https://colab.research.google.com/drive/1zi_tPMVHspjUuN3HdHcBwegxYgZyQ5LS#scrollTo=3FEg5eu51c3O
2. https://github.com/aimat-lab/gcnn_keras
3. https://github.com/tensorflow/graphics/blob/master/tensorflow_graphics/nn/layer/graph_convolution.py

In [43]:
import torch
from torch import Tensor
# from X (the input to the tensorflow model, reshape and recast as conveninet pytorch format)
# ...doing
pytorch_X = torch.tensor(X[:1].reshape(-1,12)) # the slicX[e [:1] picks up the first event
tf_tensor = tf.convert_to_tensor(pytorch_X)


In [44]:
# build a simple FCN and perform forward pass
model = MLPF_FCN()
model(tf_tensor);

TypeError: relu() missing 1 required positional argument: 'features'